# Imports

In [1]:
import sys
 
# setting path
sys.path.append('..')

In [2]:
from dataset_builder import EEDatasetBuilder
import geemap
import ee

# Map

In [3]:
Map = geemap.Map(center=[-11.8673, -76.9544], zoom=2.5)
Map

Map(center=[-11.8673, -76.9544], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

# Dataset builder

In [5]:
ee_dataset_builder = EEDatasetBuilder()

Make the biomass band with some filters

In [6]:
ee_dataset_builder.filtered_biomass_layer_from_raster(
    biomass_raster='Spawn_AGB_tCO2e',
    filter_dict={
                # 'forest_non_forest': {'date_range':['2010-01-01', '2010-12-31']},
                 # 'min_forest_age': {'age':40},
                # 'very_low_density_rural': {'year': 2010},
                # 'forest_loss': {'year': 2010, 'distance': 5000},
                # 'forest_gain' : {'distance': 5000},
                # 'roads': {'distance': 5000},
                # 'fire': {'year': 2010}
    }
)

Make covariates bands

In [7]:
ee_dataset_builder.spatial_covariates(covariates=['brazil_roads', 'terrain', 'brazil_protected_areas', 'population_density',
                                                 'forest_age', 'urbanization', 'brazil_surrounding_forest', 'brazil_pasture',
                                                 'brazil_agriculture']) # TODO: 'rivers' + 'urban_distance'

List band names of image

In [8]:
ee_dataset_builder.image.bandNames().getInfo()

['Spawn_AGB_tCO2e',
 'brazil_roads',
 'elevation',
 'aspect',
 'slope',
 'hillshade',
 'brazil_protected_areas',
 'population_density',
 'forest_age',
 'urbanization',
 'brazil_surrounding_forest',
 'brazil_pasture',
 'brazil_agriculture']

In [9]:
ee_dataset_builder.image

So the image has the added bands; it basically seems to work.

Use the below code to make a plot and confirm results of mask match what is expected for both biomass and the features.

### Visualize in map

In [10]:
biomass_vis_param = { 
  'min': 0.0,
  'max': 600.0,
  'palette': ['d9f0a3', 'addd8e', '78c679', '41ab5d', '238443', '005a32']}

Map.addLayer(ee_dataset_builder.image.select('Spawn_AGB_tCO2e'), biomass_vis_param, 'Spawn Mature Forest Biomass', True)

In [11]:
roads_vis_param = { 
  'min': 0.0,
  'max': 10000.0,
  'palette':  [
    'FFFFFF','FFD0FF','FFA2FF','FF73FF',
    'FF64FF','FF36FF','FF07FF','FF946A',
    'FFD42A','FFCE00','D2A000','4B9300',
    '168E00','005000'
  ]};

Map.addLayer(ee_dataset_builder.image.select('brazil_roads'), roads_vis_param, 'Roads distance', True)
# Map.addLayer(ee_dataset_builder.image.select('distance'), roads_vis_param, 'Rivers distance', True)

In [12]:
protected_areas_vis_param = { 
  'min': 0.0,
  'max': 200000.0,
  'palette':  [
    'FFFFFF','FFD0FF','FFA2FF','FF73FF',
    'FF64FF','FF36FF','FF07FF','FF946A',
    'FFD42A','FFCE00','D2A000','4B9300',
    '168E00','005000'
  ]};
Map.addLayer(ee_dataset_builder.image.select('brazil_protected_areas'), protected_areas_vis_param, 'Distance to Protected Areas')

In [13]:
population_density_vis_param = { 
  'min': 0.0,
  'max': 500.0,
  'palette':  [
    'FFFFFF','FFD0FF','FFA2FF','FF73FF',
    'FF64FF','FF36FF','FF07FF','FF946A',
    'FFD42A','FFCE00','D2A000','4B9300',
    '168E00','005000'
  ]};
Map.addLayer(ee_dataset_builder.image.select('population_density'), population_density_vis_param, 'Population Density')

In [14]:
forest_age_vis_param = { 
  'min': 0.0,
  'max': 100.0,
  'palette':  [
    'red','orange','yellow','green'
  ]};
Map.addLayer(ee_dataset_builder.image.select('forest_age'), forest_age_vis_param, 'Forest age')

In [15]:
urbanization_vis_param = { 
  'min': 0.0,
  'max': 7.0,
  'palette':  [
    'green','yellow','orange','red'
  ]};
Map.addLayer(ee_dataset_builder.image.select('urbanization'), urbanization_vis_param, 'Urbanization')

In [16]:
surrounding_forest_vis_param = { 
  'min': 0.0,
  'max': 700.0,
  'palette':  [
    'green','yellow','orange','red'
  ]};
Map.addLayer(ee_dataset_builder.image.select('brazil_surrounding_forest'), surrounding_forest_vis_param, 'Surrounding Forest')

In [17]:
pasture_vis_param = { 
  'min': 0.0,
  'max': 100.0,
  'palette':  [
    'red','orange','yellow','green'
  ]};
Map.addLayer(ee_dataset_builder.image.select('brazil_pasture'), pasture_vis_param, 'Surrounding Pasture')

In [18]:
agriculture_vis_param = { 
  'min': 0.0,
  'max': 100.0,
  'palette':  [
    'red','orange','yellow','green'
  ],
    'format': 'png'};
Map.addLayer(ee_dataset_builder.image.select('brazil_agriculture'), agriculture_vis_param, 'Surrounding Agriculture')

In [19]:
# urban_distance_vis_param = { 
#   'min': 0.0,
#   'max': 100000.0,
#   'palette':  [
#     'FFFFFF','FFD0FF','FFA2FF','FF73FF',
#     'FF64FF','FF36FF','FF07FF','FF946A',
#     'FFD42A','FFCE00','D2A000','4B9300',
#     '168E00','005000'
#   ]};
# Map.addLayer(ee_dataset_builder.image.select('urban_distance'), urban_distance_vis_param, 'Distance to Urban Areas')

## Export samples CSV to GCP

Check progress in your GEE console: https://code.earthengine.google.com/tasks

In [ ]:
# Gridded world shapefile asset in GEE
shp_asset_path = 'projects/ee-earthshot-shapefiles/assets/Brazil_gridded_1_degree'

# This will take quite some time
gcp_bucket = 'eartshot-science-team'
gcp_folder_name = 'deforestation_risk'
scale = 300
numPixels = 200
samples_folder_name = f'Brazil_samples_csv_scale{scale}_{numPixels}numPixels'

ee_dataset_builder.samples_csv_export(shp_asset_path,
                                      name_gcp_bucket=gcp_bucket,
                                      folder_in_gcp_bucket=gcp_folder_name + '/' + samples_folder_name,
                                      numPixels=numPixels, scale=scale)

## Export tiles to Google Drive

In [ ]:
# shp_asset_path = 'projects/ee-earthshot-shapefiles/assets/Brazil_gridded_1_degree'
# name_google_drive_folder = 'test_export_tiles_latinAm_gridded_10degrees'
# maxPixels = 1e13
# scale=100

# # This will take quite some time
# ee_dataset_builder.tiles_export(shp_asset_path, 
#                                     name_output_google_drive_folder=name_google_drive_folder, 
#                                     scale=scale, maxPixels=maxPixels)